In [1]:
import nltk
import random
import string 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [2]:

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /home/atwal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/atwal/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /home/atwal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/atwal/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
with open("panjab.txt", 'r', errors='ignore') as f:
    raw = f.read().lower()

sent_tokens= nltk.sent_tokenize(raw)
word_tokens = nltk.word_tokenize(raw)

print("Total sentences:", len(sent_tokens))
print("Total words:", len(word_tokens))

Total sentences: 7
Total words: 1521


In [4]:
from nltk.stem import WordNetLemmatizer

lemmer = nltk.stem.WordNetLemmatizer()
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))


In [6]:
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up", "hey")
THANK_INPUTS = ("thanks", "thank you", "thank you so much")
BYE_INPUTS = ("bye", "leave", "quit", "exit", "goodbye")

def detect_intent(user_input):
    if user_input in THANK_INPUTS:
        return "thank_you"
    if user_input in GREETING_INPUTS:
        return "greeting"
    if "tell me more" in user_input:
        return "tell_me_more"
    if user_input in BYE_INPUTS:
        return "exit"
    return "faq" 


In [7]:
def faq_answer(user_input):
    global last_idx, last_user_query
    sent_tokens.append(user_input)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english', token_pattern=None)
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx = vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    sent_tokens.remove(user_input)
    
    if req_tfidf == 0:
        last_idx = None
        last_user_query = None
        return None
    else:
        last_idx = idx
        last_user_query = user_input 
        return sent_tokens[idx]

In [8]:
def get_related_sentences(query, top_n=3):
    global last_related_sentences, last_related_index
    
    sent_tokens.append(query)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english', token_pattern=None)
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    
    # Get indices of top similar sentences
    sorted_indices = vals.argsort()[0][-top_n-1:-1] 
    
    sent_tokens.remove(query)
    
    # Filter out sentences with very low similarity
    related_sentences = []
    for idx in sorted_indices:
        if vals[0][idx] > 0.1: 
            related_sentences.append((idx, sent_tokens[idx]))
    
    # Sort by similarity score
    related_sentences.sort(key=lambda x: vals[0][x[0]], reverse=True)
    
    last_related_sentences = related_sentences
    last_related_index = 0
    
    return related_sentences

In [9]:
def tell_me_more():
    global last_related_sentences, last_related_index, last_user_query
    
    # If we don't have a previous query, return None
    if last_user_query is None:
        return None
    
    # If we haven't generated related sentences yet, generate them
    if not last_related_sentences:
        last_related_sentences = get_related_sentences(last_user_query, top_n=5)
        last_related_index = 1  
    
    # If we have more related sentences to show
    if last_related_index < len(last_related_sentences):
        sentence = last_related_sentences[last_related_index][1]
        last_related_index += 1
        return sentence
    else:
        # No more related information
        return None

In [10]:
def generate_response(intent, user_input):
    if intent == "greeting":
        return random.choice([
            "Hi! I'm here to help you with information about the Department of Computer Science & Applications at Panjab University. What would you like to know?",
            "Hello! Ask me anything about DCSA, Panjab University!",
            "Hey there! I can help you with queries about our Computer Science department. Fire away!"
        ])
    elif intent == "thank_you":
        return random.choice([
            "You're welcome! Feel free to ask more questions.",
            "Happy to help! Anything else you'd like to know?",
            "Glad I could help! Do you have any other questions?"
        ])
    elif intent == "tell_me_more":
        more_info = tell_me_more()
        if more_info:
            return f"Here's more information: {more_info}"
        else:
            return "I don't have additional details on this topic right now. You can ask me about something else!"
    elif intent == "faq":
        answer = faq_answer(user_input)
        if answer:
            get_related_sentences(user_input, top_n=5)
            return f"Here's what I found: {answer}\n\n(Type 'tell me more' if you want additional information on this topic)"
        else:
            return "I couldn't find relevant information. Could you try rephrasing your question or ask about something more specific like admissions, fees, courses, or placements?"
    elif intent == "exit":
        return "Goodbye! Thanks for using ChatBot — take care and have a wonderful day ahead! 👋"    
    else:
        return "I didn't quite understand that. Could you please rephrase your question?"

In [11]:
# For frontend
def response(user_input):
    ui = user_input.lower().strip()
    intent = detect_intent(ui)
    return generate_response(intent, user_input)


In [12]:
# Global variables
last_idx = None
last_user_query = None
last_related_sentences = []
last_related_index = 0

print("🤖 ChatBot: Hi! I am ChatBot 🤖. I can answer your queries about Department of Computer Science and Applications, Panjab University.")
print("You can ask me about:")
print("• Courses and programs (MCA, M.Sc., Ph.D.)")
print("• Admissions and fees")  
print("• Placements and career opportunities")
print("• Infrastructure and facilities")
print("• Faculty and research")
print("\nType 'tell me more' after any answer to get additional details!")
print("Type 'bye' to exit.\n")

if __name__ == "__main__":
    while True:
        user_response = input("You: ").lower().strip()
        intent = detect_intent(user_response)
        bot_reply = generate_response(intent, user_response)
        print("🤖 ChatBot:", bot_reply)
        
        if intent == "exit":
            break      
        
        print()


🤖 ChatBot: Hi! I am ChatBot 🤖. I can answer your queries about Department of Computer Science and Applications, Panjab University.
You can ask me about:
• Courses and programs (MCA, M.Sc., Ph.D.)
• Admissions and fees
• Placements and career opportunities
• Infrastructure and facilities
• Faculty and research

Type 'tell me more' after any answer to get additional details!
Type 'bye' to exit.

🤖 ChatBot: Here's what I found: building
  panjab university, sector 14
  chandigarh, u.t.

(Type 'tell me more' if you want additional information on this topic)



/home/atwal/.local/lib/python3.13/site-packages/sklearn/feature_extraction/text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'u', 'wa'] not in stop_words.
  warnings.warn(


🤖 ChatBot: Here's what I found: building
  panjab university, sector 14
  chandigarh, u.t.

(Type 'tell me more' if you want additional information on this topic)

🤖 ChatBot: Here's what I found: # panjab university - department of computer science & applications (dcsa)
## complete information guide

---

## *university overview*

### basic information
- *full name*: panjab university
- *established*: 1882 (originally in lahore)
- *current location*: chandigarh, india
- *type*: inter-state body corporate
- *status*: one of india's oldest and most prestigious universities

### campus details
- *location*: chandigarh (union territory)
- *campus design*: red sandstone campus designed by pierre jeanneret under le corbusier's guidance
- *move to chandigarh*: 1958-1960
- *total departments*: 74 teaching and research departments/institutes/centres
- *research chairs*: 6 independent research chairs
- *affiliated colleges*: 202 colleges across punjab and chandigarh
- *constituent colleges*: 6 c